In [ ]:
# LangChain 1.0+ Agent Setup
# %pip install langchain>=1.0.0
# %pip install langchain-core>=1.0.0
# %pip install langchain-openai
# %pip install langchain-community
# %pip install langchain-tavily  # New: for Tavily search tools
# %pip install langgraph>=1.0.0
# %pip install pypdf
# %pip install faiss-cpu

For this notebook besides the OPENAI API KEY, you will need the LANGCHAIN API KEY (if you want to use langsmith platform for evaluation) and the TAVILY API KEY. 

1. [See here for the quick setup for LANGSMITH](https://docs.smith.langchain.com/).
2. [See here for the quick setup for TAVILY API KEY](https://python.langchain.com/v0.2/docs/integrations/tools/tavily_search/).

In [1]:
import os
import getpass

# Set OPENAI API Key

import os
import getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"var: ")

_set_env("OPENAI_API_KEY")

# os.environ["OPENAI_API_KEY"] = getpass.getpass()
# os.environ["TAVILY_API_KEY"] = getpass.getpass() # you can obtain your tavily api key here: https://tavily.com/
os.environ["LANGCHAIN_TRACING_V2"] = "true"
_set_env("LANGCHAIN_API_KEY")
LANGCHAIN_ENDPOINT="https://api.smith.langchain.com"
LANGCHAIN_PROJECT="default"

Create your api key in the settings page of the [langsmith platform here](https://smith.langchain.com/settings).

> Summarized from this awesome video about the discussion of chains and agents: https://www.youtube.com/watch?v=bYLHklxEd_k&t=1801s


- Chains and agents represent two concepts in technology with distinct functionalities. Chains refer to linear sequences of events or processes that rely on the step-by-step execution of tasks, often requiring predefined inputs and producing predictable outcomes. 

- Agents, conversely, are autonomous systems capable of making decisions and performing tasks independently, adapting to new situations without explicit instructions. While chains excel in structured environments with clear rules, agents are designed to navigate complex, dynamic scenarios, learning and adjusting their behavior over time. The core difference lies in the level of autonomy and complexity each can handle, with agents typically seen as more advanced due to their self-governing nature.

# The Modern Tool Calling Way of LangChain

In [ ]:
# LangChain 1.0: Modern Tool Calling with create_agent
# The new create_agent function replaces the older create_tool_calling_agent + AgentExecutor pattern

from langchain_core.tools import tool
from langchain.agents import create_agent  # New in LangChain 1.0
from langchain_openai import ChatOpenAI

@tool
def multiply(x: float, y: float) -> float:
    """Multiply 'x' times 'y'."""
    return x * y

@tool
def exponentiate(x: float, y: float) -> float:
    """Raise 'x' to the 'y'."""
    return x**y

@tool
def add(x: float, y: float) -> float:
    """Add 'x' and 'y'."""
    return x + y

tools = [multiply, exponentiate, add]

# LangChain 1.0: create_agent - the new unified way to create agents
# Supports model string format "provider:model" or direct model instances
agent = create_agent(
    model="openai:gpt-4o",  # String format: "provider:model-name"
    tools=tools,
    prompt="You're a helpful assistant that can perform math operations.",
)

# The agent is now a LangGraph graph - invoke directly
result = agent.invoke({"messages": [{"role": "user", "content": "what's 3 plus 5 multiplied by 16?"}]})
print(result["messages"][-1].content)

In [ ]:
# LangChain 1.0: Streaming with create_agent
# The agent now supports native streaming without AgentExecutor

# Stream the response token by token
for chunk in agent.stream({"messages": [{"role": "user", "content": "what's 3 plus 5 multiplied by 16?"}]}):
    # Each chunk contains messages from that step
    if "messages" in chunk:
        for msg in chunk["messages"]:
            if hasattr(msg, 'content') and msg.content:
                print(msg.content, end="", flush=True)
    print()  # newline after each step

source: https://arc.net/l/quote/cdomnqvm

So when should you use with_structured_output versus binding tools and reading tool calls directly?

with_structured_output always returns a structured output in the schema that you specified. This is useful when you want to force the LLM to output information that matches a specific schema. This is useful for information extraction tasks.

bind_tools is more general and can select a specific tool - or no tool, or multiple tools! This is useful when you want to allow the LLM to have more flexibility in how it should respond - for example, in agent applications where you need to choose which tools to invoke but also respond to the user.

**Tool Calling Also with Pydantic**

In [ ]:
from typing import Optional

from pydantic import BaseModel, Field


# Pydantic
class Joke(BaseModel):
    """Joke to tell user."""

    setup: str = Field(description="The setup of the joke")
    punchline: str = Field(description="The punchline to the joke")
    rating: Optional[int] = Field(
        default=None, description="How funny the joke is, from 1 to 10"
    )


llm_with_structured_output = llm.with_structured_output(Joke)

In [5]:
llm_with_structured_output.invoke("Joke about how cats are evil")

Failed to use model_dump to serialize <class 'pydantic._internal._model_construction.ModelMetaclass'> to JSON: TypeError("BaseModel.model_dump() missing 1 required positional argument: 'self'")
Failed to use dict to serialize <class 'pydantic._internal._model_construction.ModelMetaclass'> to JSON: TypeError("BaseModel.dict() missing 1 required positional argument: 'self'")
Failed to use model_dump to serialize <class 'pydantic._internal._model_construction.ModelMetaclass'> to JSON: TypeError("BaseModel.model_dump() missing 1 required positional argument: 'self'")
Failed to use dict to serialize <class 'pydantic._internal._model_construction.ModelMetaclass'> to JSON: TypeError("BaseModel.dict() missing 1 required positional argument: 'self'")


Joke(setup='Why did the cat sit on the computer?', punchline='Because it wanted to keep an eye on the mouse!', rating=7)

# Tool Calling

In [6]:
from langchain_openai import ChatOpenAI

def send_error_message(message: str) -> str:
    """Send an error message to the user."""
    return f"Error: {message}"

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
llm_tools = llm.bind_tools([send_error_message])
output = llm_tools.invoke("If the following request is impossible to achieve, send an error message to the user with a funny punchline:\
    My request is that I become the master of the universe with all the money and intelligence that surpasses any human ever.")
# Pydantic

output

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_etkst4arMTR3tXEZ65L4ILDL', 'function': {'arguments': '{"message":"Becoming the master of the universe is a tall order! But hey, if you find a genie, just make sure to read the fine print—last time I checked, they don\'t cover \'universal domination\'!"}', 'name': 'send_error_message'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 58, 'prompt_tokens': 93, 'total_tokens': 151, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0392822090', 'id': 'chatcmpl-BXUnCb31Ivv2S1aZc7IF8smyGTnFo', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--5be37ae6-61d9-4c0e-8b58-311916a8edda-0', tool_calls=[{'name': 'send_error_message',

In [8]:
class send_error_message(BaseModel):
    """Send an error message to the user."""
    message: str = Field(description="The error message to send to the user")


llm_tools = llm.bind_tools([send_error_message])

In [9]:
llm_tools.invoke("If the following request is impossible to achieve, send an error message to the user with a funny punchline:\
    My request is that I become the master of the universe with all the money and intelligence that surpasses any human ever.")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_xgh6bGGGFXcQlOquAHBlo0Ry', 'function': {'arguments': '{"message":"Becoming the master of the universe is a tall order! But hey, if you find a genie, just make sure to read the fine print—last time I checked, they don\'t cover \'universal domination\' in their warranty!"}', 'name': 'send_error_message'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 62, 'prompt_tokens': 103, 'total_tokens': 165, 'completion_tokens_details': {'audio_tokens': 0, 'reasoning_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_06737a9306', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-eec081a6-fed6-421c-941e-8b9fe4e5625b-0', tool_calls=[{'name': 'send_error_message', 'args': {'message': "Becoming the master of the universe

In [7]:
from langchain_core.output_parsers import PydanticToolsParser


query = "If the following request is impossible to achieve, send an error message to the user with a funny punchline:\
    My request is that I become the master of the universe with all the money and intelligence that surpasses any human ever."

chain = llm_tools | PydanticToolsParser(tools=[send_error_message])
chain.invoke(query)

["Error: Becoming the master of the universe is a tall order! But hey, if you find a genie, just make sure to read the fine print—last time I checked, they don't cover 'universal domination' in their warranty!"]

In [8]:
from langchain_core.tools import tool


@tool
def add(a: int, b: int) -> int:
    """Adds a and b."""
    return a + b


@tool
def multiply(a: int, b: int) -> int:
    """Multiplies a and b."""
    return a * b


tools = [add, multiply]

llm_with_tools = llm.bind_tools(tools)

In [9]:
from langchain_core.messages import HumanMessage

query = "What is 3 * 12? Also, what is 11 + 49?"

messages = [HumanMessage(query)]

ai_msg = llm_with_tools.invoke(messages)

print(ai_msg.tool_calls)

messages.append(ai_msg)

[{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_JwJ8JfhOHK7n4fcgzQxHz0VQ', 'type': 'tool_call'}, {'name': 'add', 'args': {'a': 11, 'b': 49}, 'id': 'call_a4E0BvICONjAJsWcywByijnc', 'type': 'tool_call'}]


In [12]:
messages

[HumanMessage(content='What is 3 * 12? Also, what is 11 + 49?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_3knhFQcqNDuFU22zc3cBcAAB', 'function': {'arguments': '{"a": 3, "b": 12}', 'name': 'multiply'}, 'type': 'function'}, {'id': 'call_R1IysiSX4iroYgkpmELgHDrf', 'function': {'arguments': '{"a": 11, "b": 49}', 'name': 'add'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 51, 'prompt_tokens': 87, 'total_tokens': 138, 'completion_tokens_details': {'audio_tokens': 0, 'reasoning_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_06737a9306', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-4f2a3631-21e4-4439-b69c-ab1d60b80e76-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_3kn

In [10]:
for tool_call in ai_msg.tool_calls:
    selected_tool = {"add": add, "multiply": multiply}[tool_call["name"].lower()]
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)

llm_with_tools.invoke(messages)

AIMessage(content='The result of \\(3 \\times 12\\) is 36, and the result of \\(11 + 49\\) is 60.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 153, 'total_tokens': 184, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0392822090', 'id': 'chatcmpl-BXUoYXSjvsiZLuGEbdi1eAEp3g5dw', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--4ca89fd9-8cea-42c0-8338-5d75d002cf5d-0', usage_metadata={'input_tokens': 153, 'output_tokens': 31, 'total_tokens': 184, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

Making a reporting agent.

Let's build an LLM agent that can build simple reports for tasks performed just by reading through the code and files inside a folder + notes from Notion page or .txt file.

First, let's organize our thoughts, an agent will be composed of:

- Tool
- Base LLM
- Conditions for stoping and returning an output 

![](2023-10-27-19-28-10.png)

In [14]:
MODEL='gpt-4o-mini'
TEMP=0.0

In [ ]:
# LangChain 1.0: Use langchain-tavily for search tools
# The old TavilySearchResults from langchain-community is deprecated
from langchain_tavily import TavilySearch

# Create search tool
search = TavilySearch(max_results=3)

In [13]:
search = TavilySearchResults()

In [15]:
search.invoke("what is the weather in Paris?")

[{'url': 'https://en.climate-data.org/europe/france/ile-de-france/paris-44/t/may-5/',
  'content': '| Max. Temperature °C (°F) | 6.9 °C\n(44.4) °F\n| 8.1 °C\n(46.6) °F\n| 11.6 °C\n(52.9) °F\n| 15.2 °C\n(59.4) °F\n| 18.6 °C\n(65.5) °F\n| 22.1 °C\n(71.8) °F\n| 24.2 °C\n(75.6) °F\n| 24 °C\n(75.3) °F\n| 20.9 °C\n(69.6) °F\n| 16.4 °C\n(61.5) °F\n| 10.7 °C\n(51.3) °F\n| 7.5 °C\n(45.4) °F\n|\n| Precipitation / Rainfall mm (in) | 57\n(2)\n| 52\n(2)\n| 53\n(2)\n| 56\n(2)\n| 69\n(2)\n| 63\n(2)\n| 60\n(2)\n| 60\n(2)\n| 51\n(2)\n| 65\n(2)\n| 64\n(2)\n| 70\n(2)\n|\n| Humidity(%) | 85% | 81% | 76% | 71% | 71% | 68% | 65% | 66% | 71% | 79% | 86% | 86% | [...] | 17. May | 15 °C | 58 °F | 19 °C | 66 °F | 10 °C | 50 °F | 2.8 mm | 0.1 inch. |\n| 18. May | 14 °C | 57 °F | 18 °C | 65 °F | 9 °C | 48 °F | 3.4 mm | 0.1 inch. |\n| 19. May | 14 °C | 57 °F | 18 °C | 65 °F | 9 °C | 49 °F | 2.7 mm | 0.1 inch. |\n| 20. May | 14 °C | 57 °F | 18 °C | 65 °F | 9 °C | 49 °F | 4.0 mm | 0.2 inch. |\n| 21. May | 15 °C | 59

In [16]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

loader = WebBaseLoader("https://python.langchain.com/docs/get_started/introduction")
docs = loader.load()
docs

USER_AGENT environment variable not set, consider setting it to identify your requests.


[Document(metadata={'source': 'https://python.langchain.com/docs/get_started/introduction', 'title': 'Introduction | 🦜️🔗 LangChain', 'description': 'LangChain is a framework for developing applications powered by large language models (LLMs).', 'language': 'en'}, page_content='\n\n\n\n\nIntroduction | 🦜️🔗 LangChain\n\n\n\n\n\n\nSkip to main contentWe are growing and hiring for multiple roles for LangChain, LangGraph and LangSmith.  Join our team!IntegrationsAPI ReferenceMoreContributingPeopleError referenceLangSmithLangGraphLangChain HubLangChain JS/TSv0.3v0.3v0.2v0.1💬SearchIntroductionTutorialsBuild a Question Answering application over a Graph DatabaseTutorialsBuild a simple LLM application with chat models and prompt templatesBuild a ChatbotBuild a Retrieval Augmented Generation (RAG) App: Part 2Build an Extraction ChainBuild an AgentTaggingBuild a Retrieval Augmented Generation (RAG) App: Part 1Build a semantic search engineBuild a Question/Answering system over SQL dataSummarize T

In [17]:
docs[0]

Document(metadata={'source': 'https://python.langchain.com/docs/get_started/introduction', 'title': 'Introduction | 🦜️🔗 LangChain', 'description': 'LangChain is a framework for developing applications powered by large language models (LLMs).', 'language': 'en'}, page_content='\n\n\n\n\nIntroduction | 🦜️🔗 LangChain\n\n\n\n\n\n\nSkip to main contentWe are growing and hiring for multiple roles for LangChain, LangGraph and LangSmith.  Join our team!IntegrationsAPI ReferenceMoreContributingPeopleError referenceLangSmithLangGraphLangChain HubLangChain JS/TSv0.3v0.3v0.2v0.1💬SearchIntroductionTutorialsBuild a Question Answering application over a Graph DatabaseTutorialsBuild a simple LLM application with chat models and prompt templatesBuild a ChatbotBuild a Retrieval Augmented Generation (RAG) App: Part 2Build an Extraction ChainBuild an AgentTaggingBuild a Retrieval Augmented Generation (RAG) App: Part 1Build a semantic search engineBuild a Question/Answering system over SQL dataSummarize Te

In [18]:
documents = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
).split_documents(docs)

documents

[Document(metadata={'source': 'https://python.langchain.com/docs/get_started/introduction', 'title': 'Introduction | 🦜️🔗 LangChain', 'description': 'LangChain is a framework for developing applications powered by large language models (LLMs).', 'language': 'en'}, page_content='Introduction | 🦜️🔗 LangChain'),
 Document(metadata={'source': 'https://python.langchain.com/docs/get_started/introduction', 'title': 'Introduction | 🦜️🔗 LangChain', 'description': 'LangChain is a framework for developing applications powered by large language models (LLMs).', 'language': 'en'}, page_content='Skip to main contentWe are growing and hiring for multiple roles for LangChain, LangGraph and LangSmith.  Join our team!IntegrationsAPI ReferenceMoreContributingPeopleError referenceLangSmithLangGraphLangChain HubLangChain JS/TSv0.3v0.3v0.2v0.1💬SearchIntroductionTutorialsBuild a Question Answering application over a Graph DatabaseTutorialsBuild a simple LLM application with chat models and prompt templatesBui

In [19]:
vector = FAISS.from_documents(documents, OpenAIEmbeddings())

In [20]:
retriever = vector.as_retriever()

In [21]:
retriever.invoke("What is langchain?")[0]

Document(metadata={'source': 'https://python.langchain.com/docs/get_started/introduction', 'title': 'Introduction | 🦜️🔗 LangChain', 'description': 'LangChain is a framework for developing applications powered by large language models (LLMs).', 'language': 'en'}, page_content='LangChain is a framework for developing applications powered by large language models (LLMs).\nLangChain simplifies every stage of the LLM application lifecycle:')

In [22]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "langchain_search",
    "Search for information about LangChain. For any questions about LangChain, you must use this tool!",
)

In [23]:
tools = [search, retriever_tool]

In [24]:
from langchain_openai import ChatOpenAI

MODEL="gpt-4o-mini"
llm = ChatOpenAI(model=MODEL, temperature=0)

llm.invoke("Hi")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 8, 'total_tokens': 17, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0392822090', 'id': 'chatcmpl-BXUpeY1CwlOiqcfThBmaEf8dYeWxR', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--076647ce-b418-4e6d-8fbc-e52a74cdae53-0', usage_metadata={'input_tokens': 8, 'output_tokens': 9, 'total_tokens': 17, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [ ]:
# LangChain 1.0: Hub imports moved to langchain-classic
# Note: Using hub.pull() is no longer recommended in LangChain 1.0
# Instead, define prompts directly or use langchain-classic for backwards compatibility

# OLD pattern (deprecated):
# from langchain import hub
# prompt = hub.pull("hwchase17/openai-tools-agent")

# NEW pattern: Define prompts directly with ChatPromptTemplate
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant"),
    MessagesPlaceholder(variable_name="chat_history", optional=True),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])
print(prompt.messages)

In [26]:
prompt.messages[0]

SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={})

In [27]:
prompt.messages[1]

MessagesPlaceholder(variable_name='chat_history', optional=True)

In [28]:
prompt.messages[2]

HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})

In [29]:
prompt.messages[3]

MessagesPlaceholder(variable_name='agent_scratchpad')

In [30]:
tools

[TavilySearchResults(api_wrapper=TavilySearchAPIWrapper(tavily_api_key=SecretStr('**********'))),
 Tool(name='langchain_search', description='Search for information about LangChain. For any questions about LangChain, you must use this tool!', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x10b19ff60>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x118eb3bd0>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_separator='\n\n', response_format='content'), coroutine=functools.partial(<function _aget_relevant_documents at 0x10b1ec5e0>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x118eb3bd0>, search_kwargs={}), document_pro

In [ ]:
# LangChain 1.0: create_agent replaces create_tool_calling_agent
from langchain.agents import create_agent

# Create agent with search and retriever tools
agent = create_agent(
    model="openai:gpt-4o-mini",  # String format
    tools=tools,  # [search, retriever_tool]
    prompt="You are a helpful assistant. Use tools when needed.",
)

In [33]:
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: message_formatter(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMes

In [ ]:
# LangChain 1.0: Agents no longer need AgentExecutor wrapper
# The create_agent function returns a runnable graph directly

# agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)  # OLD pattern
# result = agent_executor.invoke({"input": "..."})  # OLD pattern

# NEW pattern: invoke the agent directly
result = agent.invoke({"messages": [{"role": "user", "content": "What are some useful applications you can build with LangChain? List 5 in bullet points."}]})
print(result["messages"][-1].content)

In [35]:
agent_executor.invoke({"input": "What are some useful applications you can build with LangChain? List 5 in bullet points."})



> Entering new AgentExecutor chain...

Invoking: `langchain_search` with `{'query': 'useful applications of LangChain'}`


LangChain is a framework for developing applications powered by large language models (LLMs).
LangChain simplifies every stage of the LLM application lifecycle:

Development: Build your applications using LangChain's open-source components and third-party integrations.
Use LangGraph to build stateful agents with first-class streaming and human-in-the-loop support.
Productionization: Use LangSmith to inspect, monitor and evaluate your applications, so that you can continuously optimize and deploy with confidence.
Deployment: Turn your LangGraph applications into production-ready APIs and Assistants with LangGraph Platform.



LangChain implements a standard interface for large language models and related
technologies, such as embedding models and vector stores, and integrates with
hundreds of providers. See the integrations page for
more.

Introduction | 🦜️🔗 LangC

{'input': 'What are some useful applications you can build with LangChain? List 5 in bullet points.',
 'output': 'Here are five useful applications you can build with LangChain:\n\n- **Chatbots**: Create interactive chatbots that can engage users in conversation, providing information or assistance based on user queries.\n\n- **Question Answering Systems**: Develop systems that can answer questions based on specific datasets, such as databases or documents, using natural language processing.\n\n- **Retrieval Augmented Generation (RAG) Applications**: Build applications that combine retrieval of relevant information with generative capabilities to produce contextually accurate responses.\n\n- **Semantic Search Engines**: Implement search engines that understand the context and semantics of queries, providing more relevant search results.\n\n- **Text Summarization Tools**: Create applications that can summarize large texts into concise summaries, making it easier for users to digest info

Now we can inspect what is going on in the [langsmith platform!](https://smith.langchain.com/o/b68c89e6-dae0-54ab-a84b-3201d39a3c50/settings).